# 02 - Intro to Pandas

Pandas is a popular python library that can be used for data manipulation and analysis. It can be thought of as the excel wihin python and boasts similar functionality, can do far more imppressive things.

We will cover some of the most common workflows that pandas is best suited for.

**What will be covered:**
1. pandas `.read_csv()`
2. pandas `.groupby()`
3. pandas `.groupby().agg()`
4. numpy `.where()`
5. pandas `.to_csv()`

**References:**
* Pandas documentation - [link](https://pandas.pydata.org/docs/user_guide/index.html)

In [1]:
import pandas as pd
import numpy as np

## pandas read_csv()

The best way to create a python variable from csv data

Reference: [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

In [2]:
df = pd.read_csv('data/AB_NYC_2019.csv')

In [3]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


## Grouping data with pandas

Pandas has a powerful `.groupby()` method that allows you to perform pivot table like operations on the data. 

You need to ensure that you select which columns you would like to be included in the grouping, which columns to group by, and which operation you would like to perform on the remaining columns. Here, we use `.mean()` however you can use other statistical functions such as sum, count, min, max, etc...

In this example we want to find the average price and group by neighbourhood_group. At the end, we will `.sort_values()` on price in descending prder

Reference: 
* [pandas.DataFrame.groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)
* [pandas.DataFrame.sort_values](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)

In [4]:
df[['neighbourhood_group','price']]\
    .groupby(['neighbourhood_group'])\
    .mean()\
    .sort_values('price', ascending=False)

,price
neighbourhood_group,
Manhattan,196.875814
Brooklyn,124.383207
Staten Island,114.812332
Queens,99.517649
Bronx,87.496792


Here we want to get the top 10 hosts and also see their name and neighbourhood

In [5]:
df[['host_id','host_name','neighbourhood','price']]\
    .groupby(['host_id','host_name','neighbourhood'])\
    .mean()\
    .sort_values('price', ascending=False).head(10)

,,,price
host_id,host_name,neighbourhood,
72390391,Jelena,Upper West Side,10000.0
20582832,Kathrine,Astoria,10000.0
5143901,Erin,Greenpoint,10000.0
3906464,Amy,Lower East Side,9999.0
1235070,Olson,East Harlem,9999.0
4382127,Matt,Lower East Side,9999.0
18128455,Rum,Tribeca,8500.0
7407743,Jack,Battery Park City,7500.0
156158778,Sally,Upper East Side,7061.0


## More complicated aggregation

The above example is good for when you want to perform a single statistical function across all remaining columns. If you want to perform different aggregations such as the example below wehre we want to run a count, sum and mean, you can use the `.agg()` function where you parse the column header as the key and the operation as the value to the dictionary.

References:
* [pandas.DataFrame.agg](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.agg.html)

In [6]:
df.groupby(['host_id','host_name'])\
    .agg({'price':'mean','latitude':'count', 'number_of_reviews':'sum'})\
    .sort_values('latitude', ascending=False)\
    .head(10)

,,price,latitude,number_of_reviews
host_id,host_name,,,
219517861,Sonder (NYC),253.195719,327,1281
107434423,Blueground,303.150862,232,29
30283594,Kara,277.528926,121,65
137358866,Kazuya,43.825243,103,87
16098958,Jeremy & Laura,208.958333,96,138
12243051,Sonder,213.031250,96,43
61391963,Corporate Housing,146.241758,91,417
22541573,Ken,215.436782,87,55
200380610,Pranjal,290.230769,65,1


## Perform calculations where conditions are met

We need ot use a library called numpy for this since it makes the following calculation so easy. In the data we have the price sometimes quoted for 30 days (i.e. 1 month) instead of nightly rates. So what we are going to do is divide the price by min nights where min nights are greater than or equal to 30.

Reference:
* [numpy.where](https://numpy.org/doc/stable/reference/generated/numpy.where.html)

In [7]:
df['nightly_rate'] = np.where(df.minimum_nights>=30, (df.price / df.minimum_nights), df.price)

In [8]:
df[df.minimum_nights>=30].sort_values('nightly_rate', ascending=False).head(3)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,nightly_rate
29238,22436899,1-BR Lincoln Center,72390391,Jelena,Manhattan,Upper West Side,40.77213,-73.98665,Entire home/apt,10000,30,0,NaN,NaN,1,83,333.333333
40433,31340283,2br - The Heart of NYC: Manhattans Lower East ...,4382127,Matt,Manhattan,Lower East Side,40.71980,-73.98566,Entire home/apt,9999,30,0,NaN,NaN,1,365,333.300000
30268,23377410,Beautiful/Spacious 1 bed luxury flat-TriBeCa/Soho,18128455,Rum,Manhattan,Tribeca,40.72197,-74.00633,Entire home/apt,8500,30,2,2018-09-18,0.18,1,251,283.333333


Here we return a subset of the dataframe only where host_name is Jelena

In [9]:
jelena_df = df[df['host_name']=='Jelena']

In [10]:
jelena_df

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,nightly_rate
11555,8990116,Bright and spacious 1 bedroom,5485733,Jelena,Brooklyn,Williamsburg,40.70862,-73.95432,Entire home/apt,150,2,19,2017-10-14,0.42,1,0,150.000000
29238,22436899,1-BR Lincoln Center,72390391,Jelena,Manhattan,Upper West Side,40.77213,-73.98665,Entire home/apt,10000,30,0,NaN,NaN,1,83,333.333333
45332,34680877,Cozy bedroom in a spacious apt with a backyard,55026020,Jelena,Brooklyn,Gowanus,40.66694,-73.99405,Private room,50,15,0,NaN,NaN,1,0,50.000000


In [11]:
df[df.minimum_nights>=30].groupby(['neighbourhood_group']).agg(({'price':'mean'}))

,price
neighbourhood_group,
Bronx,67.973684
Brooklyn,126.708991
Manhattan,234.471819
Queens,106.620499
Staten Island,88.684211


In [12]:
df[(df.minimum_nights>=30) & (df.neighbourhood =='Upper West Side')].groupby('neighbourhood').mean()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,nightly_rate
neighbourhood,,,,,,,,,,,
Upper West Side,1.914766e+07,7.521655e+07,40.786852,-73.97442,272.068681,35.299451,6.791209,0.368756,30.648352,234.307692,8.804594


In [13]:
df1 = df.groupby('neighbourhood').agg({'host_id':'count','host_name':'count'})

In [14]:
df1['nomatch'] = np.where(df1.host_id != df1.host_name, False, True)

In [15]:
df1['diff'] = df1.host_id - df1.host_name

In [16]:
df1[df1.nomatch==False].sort_values('diff')

,host_id,host_name,nomatch,diff
neighbourhood,,,,
Bushwick,2465,2464,False,1
Chelsea,1113,1112,False,1
Crown Heights,1564,1563,False,1
Ditmars Steinway,309,308,False,1
East Harlem,1117,1116,False,1
Highbridge,27,26,False,1
Queens Village,60,59,False,1
Upper East Side,1798,1797,False,1
Upper West Side,1971,1970,False,1


## Writing DataFrames to csv

This is easily done using `pandas.to_csv()` and all you have to do is specify where you would like the file to be written to and the name of the file.

References:
* [pandas.DataFrame.to_csv](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)

In [17]:
df1[df1.nomatch==False].sort_values('diff').to_csv('mismatch-names.csv')

Now check the current directory where this notebook is and you will find the above file.